In [2]:
import csv
import datetime
f = open("guns.csv", "r")
#This data came from 538: https://github.com/fivethirtyeight/guns-data
guns = list(csv.reader(f))



In [3]:
print(guns[0:4])

[['', 'year', 'month', 'intent', 'police', 'sex', 'age', 'race', 'hispanic', 'place', 'education'], ['1', '2012', '01', 'Suicide', '0', 'M', '34', 'Asian/Pacific Islander', '100', 'Home', '4'], ['2', '2012', '01', 'Suicide', '0', 'F', '21', 'White', '100', 'Street', '3'], ['3', '2012', '01', 'Suicide', '0', 'M', '60', 'White', '100', 'Other specified', '4']]


In [4]:
headers = guns[0]
#This section prepares my data by removing the header
print(headers)
gunlength = len(guns)
guns = guns[1:gunlength]
print(guns[:5])

['', 'year', 'month', 'intent', 'police', 'sex', 'age', 'race', 'hispanic', 'place', 'education']
[['1', '2012', '01', 'Suicide', '0', 'M', '34', 'Asian/Pacific Islander', '100', 'Home', '4'], ['2', '2012', '01', 'Suicide', '0', 'F', '21', 'White', '100', 'Street', '3'], ['3', '2012', '01', 'Suicide', '0', 'M', '60', 'White', '100', 'Other specified', '4'], ['4', '2012', '02', 'Suicide', '0', 'M', '64', 'White', '100', 'Home', '4'], ['5', '2012', '02', 'Suicide', '0', 'M', '31', 'White', '100', 'Other specified', '2']]


In [18]:
def count(rownum, data_name):
    #takes a row number from a dataset, dataset name and returns a dictionary of all the values at that row number
    dict1 = {}
    varlist = [row[rownum] for row in guns]
    for i in varlist:
        if i in dict1:
            dict1[i] =  dict1[i] + 1
        if i not in dict1:
            dict1[i] = 1
    return (dict1)

year_counts = count(1, guns)
print(year_counts)

dates = [datetime.datetime(year=int(row[1]), month = int(row[2]), day = 1) for row in guns]
print(dates[:5])
date_counts = {}
for date in dates:
#This function builds a dictionary of the dates in the gun dataset.
    if date in date_counts:
        date_counts[date] = date_counts[date] +1
    if date not in date_counts:
        date_counts[date] = 1


{'2013': 33636, '2014': 33599, '2012': 33563}
[datetime.datetime(2012, 1, 1, 0, 0), datetime.datetime(2012, 1, 1, 0, 0), datetime.datetime(2012, 1, 1, 0, 0), datetime.datetime(2012, 2, 1, 0, 0), datetime.datetime(2012, 2, 1, 0, 0)]


In [6]:
sex_counts = count(5, guns)
print(sex_counts)
race_counts = count(7, guns)
print(race_counts)

{'F': 14449, 'M': 86349}
{'Black': 23296, 'Hispanic': 9022, 'Native American/Native Alaskan': 917, 'Asian/Pacific Islander': 1326, 'White': 66237}


Most data shows that victims tend to be Male and White on an absolute level. I will now make the proportional to the size of the racial group and per 100,000 people.

In [7]:
f = open("census.csv", "r")
#census data is brought in to best adjust my values to population size
census = list(csv.reader(f))
print(census)


[['Id', 'Year', 'Id', 'Sex', 'Id', 'Hispanic Origin', 'Id', 'Id2', 'Geography', 'Total', 'Race Alone - White', 'Race Alone - Hispanic', 'Race Alone - Black or African American', 'Race Alone - American Indian and Alaska Native', 'Race Alone - Asian', 'Race Alone - Native Hawaiian and Other Pacific Islander', 'Two or More Races'], ['cen42010', 'April 1, 2010 Census', 'totsex', 'Both Sexes', 'tothisp', 'Total', '0100000US', '', 'United States', '308745538', '197318956', '44618105', '40250635', '3739506', '15159516', '674625', '6984195']]


In [8]:
mapping = {'Black':40250635, 'White':197318956, 'Hispanic':44618105, 'Native American/Native Alaskan': 3739506, 'Asian/Pacific Islander':15834141}

race_per_hundredk = {}
for key in race_counts:
    race_per_hundredk[key] = ((race_counts[key]/mapping[key])*100000)
    
print(race_per_hundredk)

{'Hispanic': 20.220491210910907, 'Native American/Native Alaskan': 24.521955573811088, 'Black': 57.8773477735196, 'White': 33.56849303419181, 'Asian/Pacific Islander': 8.374309664161762}


In [9]:
intents = [row[3] for row in guns]
races = [row[7] for row in guns]
homicide_race_counts = {}
for i, race in enumerate(races):
#provides the overall number of homicdes per race
    if intents[i] == 'Homicide' and race not in homicide_race_counts:
        homicide_race_counts[race] = 1
    if intents[i] == 'Homicide' and race in homicide_race_counts:
        homicide_race_counts[race] = homicide_race_counts[race] + 1

for key in homicide_race_counts:
#adjusts the absolute value of homicdes/race to population size and rate per 100000
    homicide_race_counts[key] = ((homicide_race_counts[key]/mapping[key])*100000)
    
print(homicide_race_counts)

{'Black': 48.47376942003524, 'Hispanic': 12.62940234687242, 'Native American/Native Alaskan': 8.744470526320857, 'Asian/Pacific Islander': 3.5366616982885275, 'White': 4.636148591826119}


When we limit the data analysis to deaths with homicide as the intent, the numbers per race group change significantly. Where whites previously were the second most likely group to die in the guns data, now they are the second least likely in the "homicide as intent data". This means that many of the deaths in the white racial category likely have suicide as the main intent, and not homicide, as suicide is the next major intent category.

Now I hope to awnser the question: Do accidental gun deaths correlate to education?
    
    

In [10]:
educationlist = [row[10] for row in guns]
homicide_education_counts = {}

for i, education in enumerate(educationlist):
#provides the overall number of accidental deaths per education level 1-5. 2 being H.S. grad, 4 being college grad. 5= N/A
    if intents[i] == 'Accidental' and education not in  homicide_education_counts:
        homicide_education_counts[education] = 1
    if intents[i] == 'Accidental' and education in homicide_education_counts:
        homicide_education_counts[education] =  homicide_education_counts[education] + 1

print(homicide_education_counts)

{'5': 29, 'NA': 14, '2': 634, '4': 147, '3': 328, '1': 493}


The above data isnt that useful alone, because like race these aren't adjusted to group size, so to alter this condition I used: https://www.census.gov/content/dam/Census/library/publications/2016/demo/p20-578.pdf

There I found that the education rate information (in % form) to build the edu_dict dictionary below. The keys are based on the gun data key of:

education -- educational status of the victim. Can be one of the following:
1 -- Less than High School
2 -- Graduated from High School or equivalent
3 -- Some College 
4 -- At least graduated from College (To compare to the census data I interpret this as graduated from bach or associate degree)
5 -- Not available

In [13]:
edu_dict = {'1':11.6, '2':29.5, '3':17.6, '4': 42.3}

adjusted_accident_rate = {}

for key in edu_dict:
    adjusted_accident_rate[key] = (homicide_education_counts[key]/edu_dict[key])
    
print(adjusted_accident_rate)



{'2': 21.491525423728813, '3': 18.636363636363633, '1': 42.5, '4': 3.4751773049645394}


It appears that adjusting for population size makes the number of homicides higher for victims with a lower education. This can be due to many reasons, but if class could be controlled for this would likely show a similar pattern as educational attainment is correlated with class. Those in low class neighborhoods with higher gun violence rates will be more likely to be victims of crime. This would recquire further study. 